# Tác vụ 2b: Tóm tắt văn bản trừu tượng

Trong sổ tay này, bạn sẽ xử lý các thách thức phát sinh trong quá trình tóm tắt tài liệu lớn – văn bản đầu vào có thể vượt quá độ dài ngữ cảnh của mô hình, dẫn đến việc tạo đầu ra ảo hoặc lỗi hết bộ nhớ.

Để giảm bớt những vấn đề trên, sổ tay này sẽ giới thiệu một kiến trúc sử dụng quy trình phân đoạn và xâu chuỗi câu lệnh với khung [LangChain] (https://python.langchain.com/docs/get_started/introduction.html). Đây là một bộ công cụ cho phép ứng dụng tận dụng các mô hình ngôn ngữ.

Bạn sẽ khám phá cách giải quyết các tình huống khi tài liệu của người dùng vượt quá giới hạn mã token. Quy trình phân đoạn sẽ chia tài liệu thành các phân đoạn theo ngưỡng độ dài ngữ cảnh, sau đó đưa vào các mô hình theo trình tự. Quá trình này sẽ liên kết câu lệnh giữa các đoạn, giữ lại bối cảnh trước đó. Bạn áp dụng cách tiếp cận này để tóm tắt bản chép lời cuộc gọi, bản chép lời cuộc họp, sách, bài viết, bài đăng blog và các nội dung liên quan khác.

## Tác vụ 2b.1: Thiết lập môi trường

Trong tác vụ này, bạn sẽ thiết lập môi trường của mình.

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## Tác vụ 2b.2: Tóm tắt văn bản dài 

### Định cấu hình LangChain với Boto3

Trong tác vụ này, bạn cần chỉ định LLM cho lớp LangChain Bedrock và có thể chuyển các đối số để suy luận.

In [ ]:
# model configuration
from langchain_aws import BedrockLLM
modelId = "amazon.titan-text-premier-v1:0"
llm = BedrockLLM(
    model_id=modelId,
    model_kwargs={
        "maxTokenCount": 2048,
        "temperature": 0,
        "topP": 1
    },
    client=bedrock_client
)

## Tác vụ 2b.3: Tải tệp văn bản có nhiều mã token

Trong tác vụ này, bạn có thể tìm thấy một tệp văn bản là [Thư từ CEO của Amazon gửi cho các cổ đông vào năm 2022] (https://www.aboutamazon.com/news/company-news/amazon-ceo-andy-jassy-2022-letter-to-shareholders) trong thư mục thư. Ô sau sẽ tải tệp văn bản và đếm số lượng mã token. Bạn sẽ thấy một cảnh báo cho biết số lượng mã token trong tệp văn bản đã vượt quá số lượng mã token tối đa cho mô hình này.

In [ ]:
#get tokens
shareholder_letter = "../letters/2022-letter.txt"

with open(shareholder_letter, "r") as file:
    letter = file.read()
    
llm.get_num_tokens(letter)

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Lưu ý:** Bạn có thể bỏ qua các cảnh báo một cách an toàn và chuyển sang ô tiếp theo.

## Tác vụ 2b.4: Chia văn bản dài thành các đoạn

Trong tác vụ này, bạn sẽ chia văn bản thành các đoạn ngắn hơn vì văn bản gốc quá dài để vừa với câu lệnh. `RecursiveCharacterTextSplitter` trong LangChain hỗ trợ chia văn bản dài thành các đoạn đệ quy cho đến khi kích thước của mỗi đoạn nhỏ hơn `chunk_size`. Văn bản sẽ được phân tách thành các đoạn bằng `separators=["\n\n", "\n"]` để tránh tách từng đoạn văn thành nhiều đoạn.

Khi sử dụng 6.000 ký tự mỗi đoạn, bạn có thể nhận được bản tóm tắt riêng cho từng phần. Số lượng mã token, hay từ tố, trong một đoạn phụ thuộc vào văn bản.

In [ ]:
#chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], chunk_size=4000, chunk_overlap=100
)

docs = text_splitter.create_documents([letter])

In [ ]:
num_docs = len(docs)

num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

print(
    f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens"
)

## Tác vụ 2b.5: Tóm tắt và kết hợp các đoạn

Trong tác vụ này, giả sử số lượng mã token trong các tài liệu khác đều như nhau để bạn có thể thực hiện được. Bạn có thể sử dụng `load_summarize_chain` của LangChain để tóm tắt văn bản. `load_summarize_chain` cung cấp ba cách tóm tắt: `stuff`, `map_reduce` và `refine`.

- `stuff`: đặt tất cả các đoạn vào một câu lệnh. Do đó, cách này sẽ đạt đến giới hạn tối đa của mã token.
- `map_reduce`: tóm tắt từng đoạn, kết hợp các bản tóm tắt, rồi tóm tắt bản tóm tắt kết hợp. Bản tóm tắt kết hợp quá lớn sẽ gây ra lỗi.
- `refine`: tóm tắt đoạn đầu tiên, rồi tóm tắt đoạn thứ hai với bản tóm tắt đầu tiên. Quá trình tương tự lặp lại cho đến khi tất cả các đoạn được tóm tắt.

Cả map_reduce và refine đều gọi LLM nhiều lần và mất thời gian để có được bản tóm tắt cuối cùng. Bạn có thể thử map_reduce tại đây.

In [ ]:
# Set verbose=True if you want to see the prompts being used
from langchain.chains.summarize import load_summarize_chain
summary_chain = load_summarize_chain(llm=llm, chain_type="map_reduce", verbose=False)

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Lưu ý:** Tùy thuộc vào số lượng tài liệu, hạn mức tỷ lệ yêu cầu của Bedrock và cài đặt thử lại được định cấu hình, chuỗi dưới đây có thể cần một chút thời gian để chạy.

In [ ]:
#invoke chain
output = ""
try:
    
    output = summary_chain.invoke(docs)

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

In [ ]:
# print output
print(output['output_text'])

Bạn đã thử nghiệm xong việc sử dụng quy trình phân đoạn và xâu chuỗi câu lệnh với khung LangChain để tóm tắt các tài liệu lớn trong khi giảm bớt vấn đề phát sinh do văn bản đầu vào dài.

### Tự thực hiện
- Thay đổi câu lệnh theo trường hợp sử dụng cụ thể của bạn và đánh giá đầu ra của các mô hình khác nhau.
- Thử nghiệm với độ dài mã token để nắm được độ trễ và khả năng phản hồi của dịch vụ.
- Áp dụng các nguyên tắc tạo câu lệnh khác nhau để có được đầu ra tốt hơn.

### Dọn dẹp

Bạn đã hoàn thành sổ tay này. Để chuyển sang phần tiếp theo của phòng thực hành, hãy làm như sau:

- Đóng tệp sổ tay này và tiếp tục với **Tác vụ 3**.